In [5]:
DATA_PATH = '/content/drive/MyDrive/customdataset'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import os
import cv2

In [6]:
CATEGORIES = os.listdir(DATA_PATH)
CATEGORIES

['MD TOYABUR RAHAMAN',
 'SHEIKH AKIB ALMAS',
 'NAFIS MUBASSHIR SHAH',
 'ABDULLAH AL SHAHRIAR']

In [7]:
plt.figure(figsize=(40,10))
i=0
for c in CATEGORIES:
    path = os.path.join(DATA_PATH,c)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        plt.subplot(4,20,i+1)
        plt.imshow(img_array)
        if i%20 == 0:
            plt.ylabel(c)
        plt.xticks([])
        plt.yticks([])
        i += 1
        if i%20 == 0:
            break

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

In [9]:
TRAIN_DATA = []
img_size = 128

for j, c in enumerate(CATEGORIES):
    path = os.path.join(DATA_PATH, c)
    label = j
    for i in tqdm(os.listdir(path)):
        img = cv2.imread(os.path.join(path, i))
        img = cv2.resize(img, (img_size, img_size))
        TRAIN_DATA.append([img, label])

print(len(TRAIN_DATA))

100%|██████████| 20/20 [00:04<00:00,  4.78it/s]

80


In [ ]:
train_imgs=[]
train_labels=[]
for img, label in TRAIN_DATA:
  train_imgs.append(img)
  train_labels.append(label)

In [ ]:
train_imgs_np=np.array(train_imgs)
train_labels_np=np.array(train_labels)
print(train_imgs_np.shape, train_labels_np.shape)

(80, 128, 128, 3) (80,)


In [ ]:
model = keras.Sequential([
    keras.Input(shape=(img_size, img_size, 3)),
    layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),  # Batch Normalization for stabilization
    layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_94 (Conv2D)          (None, 124, 124, 32)      2432      
                                                                 
 max_pooling2d_94 (MaxPooli  (None, 62, 62, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_95 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_95 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_96 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_96 (MaxPooli  (None, 14, 14, 128)     

In [ ]:
model.compile(
    optimizer= 'adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
h = model.fit(x=train_imgs_np, y=train_labels_np, epochs=15, validation_split=0.2, batch_size=32)

Epoch 1/15
2/2 [==============================] - 3s 2s/step - loss: 7.9190 - accuracy: 0.3281 - val_loss: 32.3911 - val_accuracy: 0.0000e+00
Epoch 2/15
2/2 [==============================] - 3s 1s/step - loss: 15.3793 - accuracy: 0.3750 - val_loss: 41.6636 - val_accuracy: 0.0000e+00
Epoch 3/15
2/2 [==============================] - 3s 2s/step - loss: 6.5864 - accuracy: 0.4219 - val_loss: 10.5149 - val_accuracy: 0.0000e+00
Epoch 4/15
2/2 [==============================] - 4s 2s/step - loss: 5.4501 - accuracy: 0.3906 - val_loss: 5.8371 - val_accuracy: 0.0000e+00
Epoch 5/15
2/2 [==============================] - 3s 2s/step - loss: 1.3148 - accuracy: 0.4688 - val_loss: 4.8876 - val_accuracy: 0.0000e+00
Epoch 6/15
2/2 [==============================] - 3s 1s/step - loss: 0.8101 - accuracy: 0.7188 - val_loss: 4.1072 - val_accuracy: 0.0000e+00
Epoch 7/15
2/2 [==============================] - 3s 2s/step - loss: 0.6752 - accuracy: 0.7031 - val_loss: 7.1400 - val_accuracy: 0.0000e+00
Epoch 8/1

In [ ]:
# Save the trained model
model.save('mask_detection_model.h5')
model = keras.models.load_model('mask_detection_model.h5')

In [ ]:

import cv2

import numpy as np

from tensorflow import keras

from tensorflow.keras import layers

import os



# Load your trained model here (replace 'your_model.h5' with the actual filename)



# Webcam

def detect_person():

    cap = cv2.VideoCapture(0)  # Use 0 for the default webcam

    while True:

        ret, frame = cap.read()

        if not ret:

            print("Failed to grab frame")

            break

        # Preprocess the frame

        resized_frame = cv2.resize(frame, (img_size, img_size))

        normalized_frame = resized_frame / 255.0  # Normalize pixel values

        input_data = np.expand_dims(normalized_frame, axis=0)

        # Predict using the loaded model

        prediction = model.predict(input_data)
        print("Raw Predictions:", prediction)

        # Get the label with the highest probability

        predicted_label = np.argmax(prediction)

        # Display the result on the frame

        if prediction[0, predicted_label] > 0.5:  # Adjust the threshold as needed

            name = CATEGORIES[predicted_label]

            cv2.putText(frame, f"Person: {name}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        else:

            cv2.putText(frame, "Not Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Display the frame

        cv2.imshow('Person Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):

            break

    cap.release()

    cv2.destroyAllWindows()

# Run person detection on webcam

detect_person()
